## Create a system mixing SMIRNOFF and non-SMIRNOFF-formatted force fields

### Parametrize a molecule with the Sage force field

First, we parametrize the ligand (toluene) with the SMIRNOFF-format Sage force field through the usual route to create an OpenMM `System`.

In [2]:
import mdtraj as md
import numpy as np
from openff.interchange import Interchange
from openff.units import unit
from openmm import app

from openff.toolkit import ForceField, Molecule
from openff.toolkit.utils import get_data_file_path

In [3]:
toluene_pdbfile = app.PDBFile(get_data_file_path("molecules/toluene.pdb"))
toluene_topology = Molecule.from_smiles("Cc1ccccc1").to_topology()

sage = ForceField("openff-2.0.0.offxml")

toluene = Interchange.from_smirnoff(force_field=sage, topology=toluene_topology)
toluene.positions = toluene_pdbfile.positions

In [4]:
ff14sb = ForceField("ff14sb_off_impropers_0.0.2.offxml")

# TODO: This PDB file also has water and ions, which are not yet intelligently parsed by the toolkit.
#       For now, we simply slice out the protein.
trajectory = md.load(
    get_data_file_path("systems/test_systems/T4_lysozyme_water_ions.pdb")
)

protein = trajectory.atom_slice(trajectory.topology.select("chainid 0"))
protein.save("protein.pdb")

t4_lysozyme_molecule = Molecule.from_pdb("protein.pdb")

t4_lysozyme = Interchange.from_smirnoff(
    force_field=ff14sb, topology=t4_lysozyme_molecule.to_topoplogy()
)
t4_lysozyme

TypeError: in method 'OEResidue_SetResidueNumber', argument 2 of type 'int'

In [5]:
complex_system = t4_lysozyme + toluene

NameError: name 't4_lysozyme' is not defined

In [ ]:
complex_system.to_openmm()